In [ ]:
import os
import sys

In [ ]:
# 把我們自己寫的模組的位置，加入到模組搜尋路徑之中，不然會有 import error
module_dir = os.path.join(os.path.dirname(os.getcwd()), 'modules')
if not module_dir in sys.path:
    sys.path.append(module_dir)

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import pickle

In [ ]:
# 載入計算回測指標的函式
from backtest import indicators

In [ ]:
def Breakout_strategy(df):
    # Donchian Channel
    df['20d_high'] = np.round(pd.Series.rolling(df['Close'], window=20).max(), 2)
    df['10d_low'] = np.round(pd.Series.rolling(df['Close'], window=10).min(), 2)

    has_position = False
    df['signals'] = 0
    for t in range(2, df['signals'].size):
        if df['Close'][t] > df['20d_high'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['Close'][t] < df['10d_low'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()
    return df

In [ ]:
def goldcross(df):
    df['20d'] = np.round(pd.Series.rolling(df['Close'], window=20).mean(), 2)
    df['5d'] = np.round(pd.Series.rolling(df['Close'], window=10).mean(), 2)

    has_position = False
    df['signals'] = 0
    for t in range(1, df['signals'].size):
        if df['5d'][t] > df['20d'][t] and df['5d'][t-1] <= df['20d'][t-1] and df['20d'][t-1] <= df['20d'][t]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['Close'][t] < df['20d'][t] and df['Close'][t-1] > df['20d'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()
    return df

In [ ]:
# 從哪一個檔案讀取資料
filename = '../data/twstockdata'

# 使用哪一個策略
strategy = goldcross

In [ ]:
# 讀出預先下載好的股價資料
with open(filename, 'rb') as f:
    data = pickle.load(file=f)

# 計算各支股票的回測結果
results = []

for symbol in data:
    try:
        strategy(data[symbol])
        if np.all(data[symbol]['signals']==0):
            print("Symbol:", symbol, "沒有出現買賣訊號。")
            continue
        SharpeRatio, maxdd, maxddd, finalRet = indicators(data[symbol])
        days = (data[symbol].index[-1] - data[symbol].index[0]).days
        results.append((SharpeRatio, maxdd, maxddd, finalRet, days,
                        data[symbol][data[symbol]['signals'] > 0]['signals'].sum(), symbol))
    except Exception as e:
        print("Error occurs at symbol:", symbol, "==>", e.args)

results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns','days', 'entries','symbol'])

In [ ]:
# Sorted by MaxDrawDown:
results_df.sort_values('MaxDrawDown',ascending=False).head()

In [ ]:
# Sorted by returns:
results_df.sort_values('returns',ascending=False).head()

In [ ]:
# Sorted by sharpe:
results_df.sort_values('sharpe',ascending=False).head()

In [ ]:
# Sorted by MaxDrawDownDuration:
results_df.sort_values('MaxDrawDownDuration',ascending=True).head()